# Analysing Piwik Logs

We will stream a JSON array of objects as we proceed into a LZMH compressed file.

In [14]:
var glob = require('glob');
var fs = require('fs');
var JSONStream = require('JSONStream');
var xz = require('xz');
var output = JSONStream.stringify();

# Data Selection

We select all timestamped files — we have one stat file per day.

In [15]:
var files = glob.sync('*-*-*.json');

Some utility functions.

In [16]:
var dateFromTimestamp = (ts) => new Date(ts * 1000).toISOString().split('T')[0];

In [17]:
output
    .pipe(new xz.Compressor(9))
    .pipe(fs.createWriteStream('./keywords.json.xz'));

var p = files.slice(0, 1).map(file => {
    return JSON.parse(fs.readFileSync(file))
        .filter(({ searches }) => searches !== '0')
        .map(({ idVisit, actionDetails }) => {        
          var allP = actionDetails
            .map(({ url, type, siteSearchKeyword:keyword, timestamp }) => ({
              url,
              type,
              keyword,
              date: dateFromTimestamp(timestamp),
              idVisit,
            }))
            .filter(({type, keyword}, i, allActions) => {
                const nextKeyword = allActions[i+1] ? allActions[i+1].keyword : null;
                
                return !(type === 'search' && nextKeyword && nextKeyword.indexOf(keyword) === 0);
            })
            .map((details, i, allActions) => {
              if (details.type === 'search') {
                  details.actions = [];
                  
                  allActions.slice(i+1).some(action => {
                     if (action.type !== 'search') {
                         details.actions.push(action);
                         return false;
                     }
                      
                     return true;
                  });
              }

              return details;
            })
            .filter(({ type }) => type === 'search')
            .map(details => new Promise((resolve, reject) => {
              output.write(details, (err) => err ? resolve() : reject(err))
            }));
        
            return Promise.all(allP);
        })
    });


Promise.all(p)
    .then(() => output.end())
    .catch(console.error);

[Promise] {}